In [1]:
import pdfplumber
import re

In [2]:

text = ""

with pdfplumber.open("./PDF_Files/DocumentToAnswerFrom.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

if not text.strip():
    print("Text is empty or unreadable")
else:
    print("Text has content")
    print(f"Length: {len(text)} characters")



Text has content
Length: 308 characters


In [3]:
#clean text
stripedText=[line for line in text.splitlines() if line.strip()]
textWithoutWeirdcarracters=[re.sub(r"[^a-zA-Z0-9\s.,!?:']", "", line) for line in stripedText  ]
stripedTextNoCar=[textWithoutWeirdcarracter for textWithoutWeirdcarracter in textWithoutWeirdcarracters if textWithoutWeirdcarracter.strip()]
print(stripedTextNoCar)


['CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner', 'CamScanner']


In [4]:
#split text into chunks 
fullCleanText=" ".join(stripedTextNoCar)
listOfFullText=[line for line in fullCleanText.split(" ")]
chunks=[{"id":i,"Text":" ".join(listOfFullText[chunk:chunk+200])} for i,chunk in enumerate(range(0,len(listOfFullText),150))]


In [10]:
#Embedding
from openai import OpenAI

import os
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENAI_API_KEY")
)
def embedText(txt):
	response=client.embeddings.create(
			model="text-embedding-3-small",
			input=txt
	)
	return (response.data[0].embedding)


for txt in chunks:
	txt["Embedding"]=embedText(txt["Text"])
	print(txt)
	


APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient credits. This account never purchased credits. Make sure your key is on the correct account or org, and if so, purchase more at https://openrouter.ai/settings/credits', 'code': 402}}

In [14]:
#Storing to chromaDB
import chromadb
clientChroma=chromadb.PersistentClient(path="./ChromaDB/")
collection = clientChroma.get_or_create_collection(
		name="Cours"
	)
collection.add(ids=[str(chunk["id"]) for chunk in chunks],documents=[chunk["Text"] for chunk in chunks],embeddings=[chunk["Embedding"] for chunk in chunks])
print(collection.peek())





{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'embeddings': array([[-0.00242578, -0.01785287,  0.01464623, ..., -0.00215117,
        -0.03086805,  0.02603035],
       [-0.0064105 ,  0.01208646, -0.00750674, ..., -0.00175565,
        -0.00165132,  0.00580395],
       [ 0.0021031 ,  0.00855106,  0.00197743, ..., -0.00835461,
        -0.00048461,  0.00990305],
       ...,
       [-0.04341212,  0.01483465,  0.05563587, ..., -0.01154136,
        -0.02950315,  0.03631522],
       [-0.03804267,  0.00573868,  0.04375744, ..., -0.01645089,
        -0.02196241,  0.0368232 ],
       [-0.0008782 ,  0.01337204,  0.03015192, ..., -0.02406474,
        -0.03457223,  0.03020131]]), 'documents': ["Institut Suprieur des Sciences Appliques et de Technologie de Sousse Internet ET Protocoles DNIA1 Semestre 2 Enseignante: INSAF SAGAAMA Anne universitaire: 2025  2026 Ce document doit tre complt par les notes de 1 cours ! Objectif du cours Lobjectif du ce cours est:  Avoir une vue gnrale sur le r

In [17]:
#handling user questions 
question ="what is Internet Protocoles ?"
embeddedQuestion=embedText(question)

In [18]:
#retrieve most relevent chuncks
from scipy.spatial import distance
Collection = clientChroma.get_collection(
			name="Cours"
		)
distances=[]
allData=collection.get(include=["documents","embeddings"])
for i in range(len(allData["ids"])):
    dist=distance.cosine(embeddedQuestion,allData["embeddings"][i])
    distances.append({"distance":dist,"index":allData["ids"][i]})
sortedDistances=sorted(distances,key=lambda x:x["distance"])

returnedSortedDistances=sortedDistances[:4]
returnedChunks=[allData["documents"][int(x["index"])] for x in returnedSortedDistances]
print(returnedChunks)


[": Prsentation de linternet II. Chapitre 2: Protocoles et communications rseau III. Chapitre 3: Adressage dans les Rseaux IP IV. Chapitre 4: Fragmentation aux sein des rseaux TCPIP V. Chapitre 5: Routage IP aux sein des rseaux TCPIP VI.Chapitre 6: Traduction d'adresse rseau pour IPv4: NAT VII.Chapitre 7: Protocoles et Services de la Couche Application 3 Chapitre 1: Prsentation de linternet 4 Objectifs du chapitre 1 Les objectifs du chapitre 1 sont les suivants :  Expliquer comment les rseaux affectent la faon dont nous interagissons, apprenons, travaillons et jouons  Expliquer comment les priphriques htes peuvent tre utilises comme des clients, des serveurs ou les deux  Expliquer l'utilisation des priphriques reseau  Comparer les topologies et les priphriques d'un rseau local aux priphriques et aux topologies d'un rseau tendu  Dcrire la structure de base d'Internet  Expliquer comment les rseaux locaux et tendus se connectent  Internet  Expliquer le concept d'un rseau convergent 5 Conn

In [ ]:
clientPrompt=OpenAI(base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENAI_API_PROMPT_KEY"))

systemPrompt={"role":"system","content":"""You are a helpful university tutor for software engineering students.
You will receive:
Context extracted from course documents (PDF chunks)
A student question
Rules:
-Answer ONLY using the provided context.
-If the context does not contain enough information to answer, say clearly:
"I don't know based on the provided document."
-Do not invent definitions, formulas, or facts.
-Explain in a clear student-friendly way.
-If useful, give a short example or simple explanation.
-Keep the answer well-structured (bullet points or steps when appropriate).
-If the question is vague, ask a short clarification question.
-Your goal is to help the student understand the lesson accurately and efficiently.
"""}

userPrompt={"role":"user","content":f"""Here is context extracted from a course document:
[CHUNK 1]
{returnedChunks[0]}
[CHUNK 2]
{returnedChunks[1]}
[CHUNK 3]
{returnedChunks[2]}
[CHUNK 4]
{returnedChunks[3]}
Using ONLY the context above, answer the following question:
Question: {question}"""}

response = client.chat.completions.create( 
model="gpt-5.1-chat-latest",
  messages=[systemPrompt,userPrompt],
temperature= 0, 
max_tokens=500
)
print(response.choices[0].message.content)



Based **only on the provided document**, *Internet Protocoles* (protocoles Internet) refer to:

### What are Internet Protocols?
- They are **communication rules** that allow **computers and network devices to communicate with each other over the Internet**.
- The course objective states that they help to:
  - Understand **how computers communicate**
  - Know and use the **main communication protocols used by the Internet**

### Examples of Internet Protocols mentioned in the document
The document lists several protocols studied in the course, such as:
- **IP (Internet Protocol)**  
  - IPv4 addressing  
  - IP packet formatting  
  - Fragmentation and reassembly  
  - IP routing
- **DNS (Domain Name System)**  
- **DHCP (Dynamic Host Configuration Protocol)**
- **Application-level protocols**  
  - Electronic mail  
  - File transfer
- **Security-related mechanisms**
  - Firewall (Pare-feu)

### Simple explanation
Internet protocols are like a **common language** that computers use so

In [10]:
#to download anything
# import sys
# !"{sys.executable}" -m pip install distance